In [ ]:
# es geht darum die verschiedenen Sentece Transformer anzuwenden, um heruaszufinden welche besser funktioniert
# äEs soll darum gehen die 2 Größten und best performanten miteinander zu vergleichen und eine ST die auf deutsche Daten traniert wurde

In [20]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import hnswlib
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np
import random

In [3]:
data_amazon = r'daten/amazon_datenbereinigt.xlsx'
df_amazon = pd.read_excel(data_amazon)


In [4]:
data_otto = r'daten/otto_datenbereinigt.xlsx'
df_otto = pd.read_excel(data_otto)


In [5]:
# separator = " | "  # Trennzeichen
# Amazon data
df_amazon_sentence = df_amazon.apply(lambda row: ''.join(row[col] for col in ['NAME', 'DESCRIPTION'] 
                                                          if not pd.isna(row[col])), axis=1).tolist()

In [6]:
# Otto data
df_otto_sentence = df_otto.apply(lambda row: ''.join(row[col] for col in ['NAME','DESCRIPTION'] 
                                                      if not pd.isna(row[col])), axis=1).tolist()

In [7]:
model_zweitbeste  = SentenceTransformer('all-MiniLM-L12-v2') 

config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 46.8kB/s]
README.md: 100%|██████████| 10.7k/10.7k [00:00<00:00, 6.86MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 50.0kB/s]
config.json: 100%|██████████| 573/573 [00:00<00:00, 399kB/s]
pytorch_model.bin: 100%|██████████| 134M/134M [00:04<00:00, 32.7MB/s] 
tokenizer_config.json: 100%|██████████| 352/352 [00:00<00:00, 369kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.17MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 68.8MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 141kB/s]


In [8]:
df_amazon_embeddings = model_zweitbeste.encode(df_amazon_sentence, batch_size = 32)

In [11]:
df_otto_embeddings = model_zweitbeste.encode(df_otto_sentence, batch_size = 32)

In [13]:
# Modell Laden https://aclanthology.org/2022.bucc-1.9.pdf quelle !!!
# zwei beste model im vergleich 
dim = df_otto_embeddings.shape[1]  # Vektor-Dimension
index = hnswlib.Index(space='l2', dim=dim)# l2 für Euklid

index.init_index(max_elements=len(df_otto), ef_construction=200, M=16)

index.add_items(df_otto_embeddings)

index.save_index('_2bestemodell_hnswlib_index_otto.bin')
index.set_ef(100) 

similar_items = []
for i, embedding in enumerate(df_amazon_embeddings):
    # Abfragen der fünf nächsten Nachbarn (das eigentliche Produkt eingeschlossen 1+)
    labels, distances = index.knn_query(embedding, k=5)
    # Speichern der ähnlichsten Produkte
    similar_products = [df_otto_sentence[j] for j in labels[0]]
    similar_items.append(similar_products)

df = pd.DataFrame(similar_items, columns=['amazon_product',
                                          'otto_product_1', 
                                          'otto_product_2',
                                          'otto_product_3',
                                          'otto_product_4'
                                          ])
df.to_excel("daten_sentence_tranformer_im_vergleich/all_distil_robertav1_2beste_amazon_otto_daten.xlsx")
df.head(10)

,amazon_product,otto_product_1,otto_product_2,otto_product_3,otto_product_4
0,schöffel fleecejacke fleece jacke pelham megal...,dare2b outdoorjacke ascension jacket wasserdic...,hummel fleecejackein dieser robusten hmlauthen...,schöffel skijacke herren skijacke ski jacket t...,schoeffel outdoorjackedie winddichte jacke 3in...
1,adidas originals outdoorjacke hooded premium l...,adidas five ten wanderschuhdie adidas terrex a...,adidas sportswear winterjacke bsc 3 streifen h...,adidas performance regenjacke utilitas rainrdy...,adidas sportswear winterjacke herren daunenwes...
2,brax stretch hose styleleeeine sportive hose m...,aatrx schlaghose sexy damen yogahose mit niedr...,ellesse jogginghose hose ellesse queenstown 1 ...,cecil dehnbund hose elastikbund mit tunnelzugc...,eevi stoffhose eevi hose ceremony graudie eleg...
3,shirtracer t shirt vorsicht vor dem frauchen p...,dogo go home sneaker vegandieser trendige snea...,dogo where am i sneaker vegandieser zauberhaf...,tini shirts print shirt pit bull hunde kinder ...,shirtracer t shirt gästebuch 70 geburtstag gäs...
4,vidaxl pouf gartenstuhl auflagen 4 stk bedruck...,meisemöbel polsterbett polsterbett frieda mit ...,vidaxl bettgestell massivholzbett mit kopfteil...,vidaxl bettgestell bettgestell blau 120x200 cm...,ormromra kletterseiloutdoor seil durchmesser s...
5,f4nt4stic t shirt star wars the mandalorian di...,f4nt4stic t shirt long cut t shirt star wars t...,star wars t shirthochwertiges t shirt von der ...,star wars t shirthochwertiges t shirt von der ...,star wars print shirt star wars han solo t shi...
6,cmp strickfleecejacke cmp hybrid jacket fix ho...,klättermusen outdoorjacke glaser jacket menrob...,cmp fahrradjacke woman jacketextraleichte fahr...,mfh langarmshirt sweatshirt schwarz pc 340gm m...,tatonka softshelljacke jesper hooded jacket me...
7,gloria hundeleine hundeleine gloria schwarz 1 ...,almgwand trachtenshirt langarmshirt geisleralm...,kerbl draht weidezaunseil elektroseil profi pe...,ulla popken schlupfhose hose sophie karomuster...,gant langarmhemd karohemddas gant herren hemd ...
8,erima sportsocken 3er pack 5 c socksmit der 5 ...,100% sportsockenphochwertiges sockenpaar im 10...,reebok classic sportsocken classic footwear in...,soliver 78 jeans ankle jeans izabell skinny f...,erima trainingsshortserima premium one 20 shor...
9,ulla popken langarmshirt shirt classic crepe k...,ambria 34 arm shirt shirt 1 tlgdas shirt wirkt...,uana langarmshirt einfarbiges lockeres langarm...,we fashion t shirt 1 tlggesteiftes rollkragens...,fldhood t shirt damen casual off shoulder slim...


In [14]:
def generate_negative_examples(df, num_negatives_per_positive=1):
    """
    Ähnliche Anzahl von negativen Beispielen, basierend auf den vorhandenen positiven Beispielen genrerien.
    Jedes negative Beispiel ist ein Paar von Produkten, die nicht übereinstimmen.
    
    :param df: DataFrame mit den positiven Beispielen.
    :param num_negatives_per_positive: Anzahl der negativen Beispiele, die pro positivem Beispiel generiert werden sollen.
    :return: DataFrame mit den negativen Beispielen.
    """
    negative_examples = []
    product_names = df['amazon_product_name'].tolist() + df['otto_variation_name'].tolist()
    
    for _, row in df.iterrows(): # der index selbst wird ignoriert 
        for _ in range(num_negatives_per_positive):
            # zuffälig zwei Produkte nehmen, die nicht paare sind 
            while True:
                product1 = random.choice(product_names)
                product2 = random.choice(product_names)
                
                # hierit wird sichergestellt, dass product1 und product2 nicht übereinstimmen und nicht in den positiven Beispielen sind
                if product1 != product2 and not ((df['amazon_product_name'] == product1) & (df['otto_variation_name'] == product2)).any():
                    break
            
            negative_examples.append({'amazon_product_name': product1, 'otto_variation_name': product2})
    
    return pd.DataFrame(negative_examples)

# schwere Negative bsp bauen!!!


In [30]:
import pandas as pd
import random

def generate_harder_negative_examples(df, num_negatives_per_positive=1):
    """
    Generiert eine ähnliche Anzahl von 'schweren' negativen Beispielen basierend auf den vorhandenen positiven Beispielen.
    'Schwere' negative Beispiele sind solche, bei denen die Produkte ähnlich, aber nicht identisch sind.
    
    :param df: DataFrame mit den positiven Beispielen und zusätzlichen Informationen wie Kategorien.
    :param num_negatives_per_positive: Anzahl der negativen Beispiele pro positivem Beispiel.
    :return: DataFrame mit den negativen Beispielen.
    """
    negative_examples = []
    categories = df['kategorie_bezeichnung'].unique()  # Eindeutige Kategorien extrahieren

    for _, row in df.iterrows():
        for _ in range(num_negatives_per_positive):
            # Wählen Sie eine Kategorie zufällig aus, die dem aktuellen positiven Beispiel entspricht
            current_category = row['kategorie_bezeichnung']
            
            # Produkte innerhalb der gleichen Kategorie, aber nicht das gleiche Produkt
            similar_products = df[(df['kategorie_bezeichnung'] == current_category) & 
                                  (df['amazon_product_name'] != row['amazon_product_name']) &
                                  (df['otto_variation_name'] != row['otto_variation_name'])]
            
            # Wenn es ähnliche Produkte gibt, wählen Sie eines zufällig als negatives Beispiel
            if not similar_products.empty:
                negative_example = similar_products.sample(1)  # Wählen Sie ein zufälliges ähnliches Produkt
                product1 = row['amazon_product_name']
                product2 = negative_example['otto_variation_name'].values[0]  # Nehmen Sie den Namen des Otto-Produkts
                
                negative_examples.append({'amazon_product_name': product1, 'otto_variation_name': product2})

    return pd.DataFrame(negative_examples)


In [16]:
df= pd.read_pickle(r'daten/otto_amazon_variation_pairs.pkl', compression='gzip')

In [32]:
positive_embeddings_1 = model_zweitbeste.encode(df['amazon_product_name'].tolist())
positive_embeddings_2 = model_zweitbeste.encode(df['otto_variation_name'].tolist())
# Absolute Differenz als Features berechnen 
positive_features = np.abs(positive_embeddings_1 - positive_embeddings_2)

In [33]:
negative_df = generate_harder_negative_examples(df)
negative_embeddings_1 = model_zweitbeste.encode(negative_df['amazon_product_name'].tolist()) #eventuell nochmal mit batchsize machen
negative_embeddings_2 = model_zweitbeste.encode(negative_df['otto_variation_name'].tolist())


In [34]:
negative_features = np.abs(negative_embeddings_1 - negative_embeddings_2)
# Features und Labels zusammenführen Positive Features
positive_labels = np.ones(len(positive_features))
negative_labels = np.zeros(len(negative_features))
# Features und Labels zusammenführen
X = np.concatenate([positive_features, negative_features], axis=0)
y = np.concatenate([positive_labels, negative_labels], axis=0)

# Trainings- und Testsets aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)

print(y_train.shape)

""" # SVM trainieren
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Vorhersagen und Evaluation 
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred)) """


(101269,)
              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91     12556
         1.0       0.92      0.90      0.91     12762

    accuracy                           0.91     25318
   macro avg       0.91      0.91      0.91     25318
weighted avg       0.91      0.91      0.91     25318



In [37]:
from sklearn import tree

clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_train, y_train)
y_pred = clf_tree.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.75      0.74      0.75     12556
         1.0       0.75      0.76      0.75     12762

    accuracy                           0.75     25318
   macro avg       0.75      0.75      0.75     25318
weighted avg       0.75      0.75      0.75     25318



In [43]:
from sklearn.linear_model import SGDClassifier

clf_GDC = SGDClassifier(loss="modified_huber", max_iter=3000, tol=1e-3)
clf_GDC.fit(X_train, y_train)
y_pred = clf_GDC.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91     12556
         1.0       0.94      0.88      0.91     12762

    accuracy                           0.91     25318
   macro avg       0.91      0.91      0.91     25318
weighted avg       0.91      0.91      0.91     25318



In [45]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()
clf_rf  = clf_rf.fit(X_train, y_train)
y_pred = clf_rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90     12556
         1.0       0.96      0.82      0.88     12762

    accuracy                           0.89     25318
   macro avg       0.90      0.89      0.89     25318
weighted avg       0.90      0.89      0.89     25318



In [35]:
# Modell speichern
from joblib import dump
dump(clf, 'modell/zweite_svm_modell_hard_Negativ_all-MiniLM-L12-v2.joblib')


['modell/zweite_svm_modell_hard_Negativ_all-MiniLM-L12-v2.joblib']

In [ ]:
'''--------------------------------------------------------------------------------------------------------------------------------------'''

In [25]:
model_finetuent_german = SentenceTransformer('all-distilroberta-v1')

In [26]:
# Weitere Modell um Sentence transformer modelle miteinander anhand der Ergebnisse zu vergleichen: auf deutsche Daten fine tuent 
df_amazon_embeddings = model_finetuent_german.encode(df_amazon_sentence, batch_size = 32)

In [29]:
df_otto_embeddings = model_finetuent_german.encode(df_otto_sentence, batch_size = 32)


KeyboardInterrupt: 

In [ ]:

dim = df_otto_embeddings.shape[1]  # Vektor-Dimension
index = hnswlib.Index(space='l2', dim=dim)# l2 für Euklid

index.init_index(max_elements=len(df_otto), ef_construction=200, M=16)

index.add_items(df_otto_embeddings)

index.save_index('_deutsche_bestemodell_hnswlib_index_otto.bin')
index.set_ef(100) 

similar_items = []
for i, embedding in enumerate(df_amazon_embeddings):
    # Abfragen der fünf nächsten Nachbarn (das eigentliche Produkt eingeschlossen 1+)
    labels, distances = index.knn_query(embedding, k=5)
    # Speichern der ähnlichsten Produkte
    similar_products = [df_otto_sentences[j] for j in labels[0]]
    similar_items.append(similar_products)

df = pd.DataFrame(similar_items, columns=['amazon_product',
                                          'otto_product_1', 
                                          'otto_product_2',
                                          'otto_product_3',
                                          'otto_product_4'
                                          ])
df.to_excel("daten_sentence_tranformer_im_vergleich/T-Systems-onsite_deutschmodell_amazon_otto_daten.xlsx")
df.head(10)

In [ ]:
# Präziste Model! Quelle : https://www.sbert.net/docs/pretrained_models.html
model_beste = sentence_transformers('sentence-transformers/all-mpnet-base-v2')